# Basics of NLP

NLP is a 2 part problem and is a useful way to deal with verbal data, which the world is full of
* The first part: Process the data into a form that a computer can understand. This usually involves elements of data cleaning and feature extraction. We look at features such as word frequency, meaning, abd grammar. A simple example is any binary feature that indicates the presence of a word

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
import nltk
# Launch the installer to download "gutenberg" and "stop words" corpora.
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
# Import the data we just downloaded and installed.
from nltk.corpus import gutenberg, stopwords

# Grab and process the raw data.
print(gutenberg.fileids())

persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# Print the first 100 characters of Alice in Wonderland.
print('\nRaw:\n', alice[0:100])

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']

Raw:
 [Alice's Adventures in Wonderland by Lewis Carroll 1865]

CHAPTER I. Down the Rabbit-Hole

Alice was


In [4]:
# This pattern matches all text between square brackets.
pattern = "[\[].*?[\]]"
persuasion = re.sub(pattern, "", persuasion)
alice = re.sub(pattern, "", alice)

# Print the first 100 characters of Alice again.
print('Title removed:\n', alice[0:100])

Title removed:
 

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on


In [5]:
# Now we'll match and remove chapter headings.
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

# Ok, what's it look like now?
print('Chapter headings removed:\n', alice[0:100])

Chapter headings removed:
 



Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothin


In [6]:
# Remove newlines and other extra whitespace by splitting and rejoining.
persuasion = ' '.join(persuasion.split())
alice = ' '.join(alice.split())

# All done with cleanup? Let's see how it looks.
print('Extra whitespace removed:\n', alice[0:100])

Extra whitespace removed:
 Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to


# So far
What we've done is use regular expression and the _sub_ method (substitute) to get rid of unnecessary information. Next, we can start to extract information from the raw text that's left over.

# So what can we extract from this?

## Tokens

Each individual, meaningful piece of information from text data is called a _token_ and the process of breaking up the raw text into these meaningful pieces is called _tokenization_. Tokens are usually words or punctuation. Punctuation can sometimes be uninfromative so we might want to remove them from analysis some time. One potentially uninformative class of tokens is called _stop words_. Think of words like "of" and "the". These tokens are very common and depending on the situation, mgiht not be particularly useful. Some approaches do, however, keep them because they add context, ex: "master of the universe" carries a different meaning as a token as opposed to "master" and "universe" separately.

In [7]:
# Here is a list of the stopwords identified by NLTK.
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
import spacy
nlp = spacy.load('en')

# All the processing work is done here, so it may take a while.
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [9]:
# Let's explore the objects we've built.
print("The alice_doc object is a {} object.".format(type(alice_doc)))
print("It is {} tokens long".format(len(alice_doc)))
print("The first three tokens are '{}'".format(alice_doc[:3]))
print("The type of each token is {}".format(type(alice_doc[0])))

The alice_doc object is a <class 'spacy.tokens.doc.Doc'> object.
It is 34430 tokens long
The first three tokens are 'Alice was beginning'
The type of each token is <class 'spacy.tokens.token.Token'>


In [10]:
from collections import Counter

# Utility function to calculate how frequently words appear in the text.
def word_frequencies(text, include_stop=True):
    
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            words.append(token.text)
            
    # Build and return a Counter object containing word counts.
    return Counter(words)
    
# The most frequent words:
alice_freq = word_frequencies(alice_doc).most_common(10)
persuasion_freq = word_frequencies(persuasion_doc).most_common(10)
print('Alice:', alice_freq)
print('Persuasion:', persuasion_freq)

Alice: [('the', 1524), ('and', 796), ('to', 724), ('a', 611), ('I', 534), ('it', 524), ('she', 508), ('of', 499), ('said', 453), ('Alice', 394)]
Persuasion: [('the', 3120), ('to', 2775), ('and', 2738), ('of', 2563), ('a', 1529), ('in', 1346), ('was', 1329), ('had', 1177), ('her', 1159), ('I', 1121)]


In [11]:
# Use our optional keyword argument to remove stop words.
alice_freq = word_frequencies(alice_doc, include_stop=False).most_common(10)
persuasion_freq = word_frequencies(persuasion_doc, include_stop=False).most_common(10)
print('Alice:', alice_freq)
print('Persuasion:', persuasion_freq)

Alice: [('I', 534), ('said', 453), ('Alice', 394), ("n't", 215), ("'s", 190), ('little', 124), ('The', 102), ('like', 84), ('went', 83), ('know', 83)]
Persuasion: [('I', 1121), ('Anne', 497), ("'s", 485), ('She', 326), ('Captain', 297), ('Mrs', 291), ('Elliot', 288), ('Mr', 255), ('He', 225), ('Wentworth', 217)]


In [12]:
# Pull out just the text from our frequency lists.
alice_common = [pair[0] for pair in alice_freq]
persuasion_common = [pair[0] for pair in persuasion_freq]

# Use sets to find the unique values in each top ten.
print('Unique to Alice:', set(alice_common) - set(persuasion_common))
print('Unique to Persuasion:', set(persuasion_common) - set(alice_common))

Unique to Alice: {'know', 'The', "n't", 'went', 'said', 'little', 'like', 'Alice'}
Unique to Persuasion: {'Mrs', 'Wentworth', 'She', 'Elliot', 'Mr', 'Captain', 'He', 'Anne'}


## Lemma

So far, we've looked at individual words. But what happens with words like think, thought, thinking etc. They all represent the same idea and it's not prudent to have different vectors representing the same idea. So we can use the root of words as another form of feature extraction. These roots are referred to as _lemma_. Here, we are making features that represent _concepts_ as opposed to _words_. 

In addition to looking at lemmas, we could perform a similar analysis and pull out prefixes (token.prefix_) or suffixes (token.suffix_).

In [13]:
# Utility function to calculate how frequently lemas appear in the text.
def lemma_frequencies(text, include_stop=True):
    
    # Build a list of lemas.
    # Strip out punctuation and, optionally, stop words.
    lemmas = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            lemmas.append(token.lemma_)
            
    # Build and return a Counter object containing word counts.
    return Counter(lemmas)

# Instantiate our list of most common lemmas.
alice_lemma_freq = lemma_frequencies(alice_doc, include_stop=False).most_common(10)
persuasion_lemma_freq = lemma_frequencies(persuasion_doc, include_stop=False).most_common(10)
print('\nAlice:', alice_lemma_freq)
print('Persuasion:', persuasion_lemma_freq)

# Again, identify the lemmas common to one text but not the other.
alice_lemma_common = [pair[0] for pair in alice_lemma_freq]
persuasion_lemma_common = [pair[0] for pair in persuasion_lemma_freq]
print('Unique to Alice:', set(alice_lemma_common) - set(persuasion_lemma_common))
print('Unique to Persuasion:', set(persuasion_lemma_common) - set(alice_lemma_common))


Alice: [('-PRON-', 758), ('say', 476), ('alice', 396), ('be', 254), ('not', 231), ('go', 133), ('think', 131), ('little', 126), ('the', 109), ('look', 105)]
Persuasion: [('-PRON-', 2241), ('anne', 497), ("'s", 466), ('captain', 303), ('elliot', 295), ('mrs', 291), ('good', 289), ('know', 258), ('think', 256), ('mr', 255)]
Unique to Alice: {'say', 'look', 'not', 'little', 'alice', 'go', 'the', 'be'}
Unique to Persuasion: {'good', 'anne', 'know', 'mr', "'s", 'mrs', 'captain', 'elliot'}


## Sentences

Beyond individual words, text can also be considered at the level of sentences. Using punctuation cues, we can split up text into sentences. Each sentence can then be summarized by, for example, using sentiment analysis to categorize sentences as having positive or negative sentiment. We may also be interested in how long sentences tend to be, and how many unique words make up a sentence.  The sentence also provides _context_ for the individual words, allowing us to draw even more information from each word.

We get a lot of automatic sentence-level information from spaCy. The `doc.sents` property will give us each sentence as a [span](https://spacy.io/docs/api/span) object. Let's look at some of that.

In [14]:
# Initial exploration of sentences.
sentences = list(alice_doc.sents)
print("Alice in Wonderland has {} sentences.".format(len(sentences)))

example_sentence = sentences[2]
print("Here is an example: \n{}\n".format(example_sentence))

Alice in Wonderland has 1678 sentences.
Here is an example: 
There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, 'Oh dear!



In [15]:
# Look at some metrics around this sentence.
example_words = [token for token in example_sentence if not token.is_punct]
unique_words = set([token.text for token in example_words])

print(("There are {} words in this sentence, and {} of them are"
       " unique.").format(len(example_words), len(unique_words)))

There are 29 words in this sentence, and 25 of them are unique.


## Parts of speech, dependencies, entities
Tokens within each sentence are also coded with the parts of speech they play. This is useful for distinguishing between _homographs_, words with the same spelling but different meaning (the umbrella term for this kind of linguistic feature is _polysemy_).  For example, the word "break" is a noun in "I need a break" but a verb in "I need to break the glass".

In [16]:
print(nlp("I need a break")[3].pos_)
print(nlp("I need to break the glass")[3].pos_)

NOUN
VERB


In [17]:
# View the part of speech for some tokens in our sentence.
print('\nParts of speech:')
for token in example_sentence[:9]:
    print(token.orth_, token.pos_)


Parts of speech:
There ADV
was VERB
nothing NOUN
so ADV
VERY ADV
remarkable ADJ
in ADP
that DET
; PUNCT


In [18]:
# View the dependencies for some tokens.
print('\nDependencies:')
for token in example_sentence[:9]:
    print(token.orth_, token.dep_, token.head.orth_)


Dependencies:
There expl was
was ROOT was
nothing attr was
so advmod remarkable
VERY advmod remarkable
remarkable amod nothing
in prep nothing
that pobj in
; punct was


Finally, spaCy gives us access to the named entities with `.ents`. In the example below you'll see some errors creep in – we can see that the entity identification rules in spaCy assume that, if it doesn't fall under any other obvious rule, any word or phrase IN ALL CAPS is an organization (if a noun) or an event (if a verb).

In [21]:
# Extract the first ten entities.
entities = list(alice_doc.ents)[0:10]
for entity in entities:
    print(entity.label_, ' '.join(t.orth_ for t in entity))

PERSON Alice
DATE the hot day
PERSON Alice
PRODUCT Rabbit
PRODUCT Rabbit
PRODUCT WAISTCOAT - POCKET
PERSON Alice
PERSON Alice
PERSON Alice
ORDINAL First


In [22]:
# All of the uniqe entities spaCy thinks are people.
people = [entity.text for entity in list(alice_doc.ents) if entity.label_ == "PERSON"]
print(set(people))

{'Begin', 'Sha', '--or', 'Fish-Footman', 'indeed:--', 'YOURS', 'Soup of the evening', 'Turn', 'M--', 'Fifteenth', 'the Duchess', 'Brandy', "the King: '", 'Fetch', 'the White Rabbit', 'Morcar', 'Lacie', "the Mock Turtle: '", 'm--', 'Run', 'Majesty', 'Edwin', 'Rabbit', 'the Lobster Quadrille', 'Prizes', 'Sentence', 'Soo', 'Bill', 'Down', 'Turtle Soup', 'Canary', 'Crab', 'Longitude', 'Pat', 'Tillie', 'Mabel', 'Stand', 'WILLIAM', 'HAD', 'Sixteenth', 'Cheshire Puss', 'Beautiful Soup', 'The White Rabbit', 'the King', 'Seaography', 'Stolen', 'Soles', 'Alice', 'Gryphon', "Don't", 'Panther', 'Latin Grammar', 'Ou', 'Serpent', 'Shy', 'Tut', "Dinah'll", 'Curiouser', 'the Queen of Hearts', 'a Lobster Quadrille', 'Treacle', 'The Queen', 'Footman', 'Elsie', 'the Lobster Quadrille?', 'Queen', 'Soup', 'the March Hare', 'Mock Turtle', 'William the Conqueror', 'Drink', 'Repeat', 'Latitude', 'The Mock Turtle', 'the Mock Turtle', 'INSIDE', 'Adventures', 'The Fish-Footman', 'Frog-Footman', 'Edgar Atheling',